![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx  `meta-llama/llama-2-70b-chat` and `ibm/granite-13b-instruct-v1` to analyze car rental customer satisfaction from text

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.
- กรุณาใช้ Projects และข้อมูล ที่เตรียมไว้ให้ใน watsonx


## Notebook content

This notebook contains the steps and code to demonstrate support of text sentiment analysis in Watsonx. It introduces commands for data retrieval, model testing and scoring.

แบบฝึกหัดนี้ประกอบไปด้วยโค้ดและขั้นตอนการใช้งาน WatsonX เพื่อการทำ sentiment analysis
มีการสอนใช้ คำสั่งเพื่อการ ดึงเดต้า เทสโมเดล และการให้คะแนนโมเดล 

Some familiarity with Python is helpful. This notebook uses Python 3.10.
แบบฝึกหัดนี้ใช้ Python 3.10 เหมาะสำหรับผู้มีพื้นฐาน Python

## Learning goal

The goal of this notebook is to demonstrate how to use `meta-llama/llama-2-70b-chat` and  `ibm/granite-13b-instruct-v1` model to analyze customer satisfaction from text.

เป้าหมายของแบบฝึกหัดนี้คือ เพื่อสาธิตวิธีการใช้โมเดล `meta-llama/llama-2-70b-chat` และ `ibm/granite-13b-instruct-v1` เพื่อวิเคราะห์ความพึงพอใจของลูกค้าจากข้อความ

## Contents

This notebook contains the following parts:
แบบฝึกหัดนี้ประกอบไปด้วย:

- [Setup](#setup)
- [Data loading](#data)
- [Foundation Models on Watsonx](#models)
- [Model testing](#predict)
- [Score](#score)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

ก่อนที่ทุกท่านจะเริ่มทำแบบฝึกหัดนี้ กรุณาทำการติดตั้ง ตามวิธีการนี้ก่อน:


-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">here</a>).


-  สร้าง <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (สามารถใช้แบบไม่มีค่าใช้จ่าย และรายละเอียด การติดตั้งเป็นดังนี้ <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">ข้อมูลเพิ่มเติม</a>).


### Install and import the `datasets` and dependecies
### ดาวน์โหลด `datasets` และติดตั้งแพคเกจที่เกี่ยวข้อง

In [20]:
%pip install wget | tail -n 1
%pip install datasets | tail -n 1
%pip install scikit-learn | tail -n 1
%pip install "ibm-watson-machine-learning>=1.0.326" | tail -n 1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Defining the WML credentials
This cell defines the WML credentials required to work with watsonx Foundation Model inferencing.

ขั้นตอนต่อไปนี้จะเป็นการระบุ credentials ที่จำเป็นต้องใช้ในการ ใช้งาน watsonx Foundation Model

**Action:** Provide the IBM Cloud user API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

**Action:** กรอกข้อมูล IBM cloud user API key รายละเอียดและวิธีการอยู่ข้างล่าางนี้
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [21]:
import os
import json
from dotenv import load_dotenv
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

### Defining the project id
The Foundation Model requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.


กำหนด project id เพื่อให้สามารถเชื่อมต่อแบบฝึกหนัดนี้ไปหา watsonx ได้อย่างถูกต้อง

In [22]:
#config Watsonx.ai environment
load_dotenv()
project_id = os.environ["PROJECT_ID"]
ibm_cloud_url = os.environ["IBM_CLOUD_URL"]
api_key = os.environ["API_KEY"]


In [23]:
#config Watsonx.ai environment

if api_key is None or ibm_cloud_url is None or project_id is None:
    raise Exception("Ensure you copied the .env file that you created earlier into the same directory as this notebook")
else:
    credentials = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }

<a id="data"></a>
## Data loading

Download the `car_rental_training_data` dataset. The dataset provides insight about customers opinions on car rental. It has a label that consists of values: unsatisfied, satisfied.

ดาวน์โหลดชุดข้อมูล `car_rental_training_data` ชุดข้อมูลนี้ให้ข้อมูลความคิดเห็นของลูกค้าเกี่ยวกับการเช่ารถ มีข้อมูลป้ายกำกับที่ประกอบด้วยค่า: ไม่พอใจ, พอใจ.


In [24]:
import wget
import pandas as pd

filename = 'car_rental_training_data.csv'
url = 'https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/data/cars-4-you/car_rental_training_data.csv'

if not os.path.isfile(filename): 
    wget.download(url, out=filename)

df = pd.read_csv("car_rental_training_data.csv", sep=';')
data = df[['Customer_Service', 'Satisfaction']]

Examine downloaded data.

In [25]:
data.head()

,Customer_Service,Satisfaction
0,I thought the representative handled the initi...,0
1,I have had a few recent rentals that have take...,0
2,car cost more because I didn't pay when I rese...,0
3,I didn't get the car I was told would be avail...,0
4,If there was not a desired vehicle available t...,1


Prepare train and test sets.

In [55]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)
comments = list(test.Customer_Service)
satisfaction = list(test.Satisfaction)

<a id="models"></a>
## Foundation Models on `watsonx.ai`

#### List available models

All avaliable models are presented under ModelTypes class.
For more information refer to [documentation](https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html#ibm_watson_machine_learning.foundation_models.utils.enums.ModelTypes).

Model ประเภท LLM ทั้งหมด สามารถดูได้จาก ModelTypes
สามารถค้นหาข้อมูลเพิ่มเติมได้จาก [documentation](https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html#ibm_watson_machine_learning.foundation_models.utils.enums.ModelTypes).


In [56]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2', 'STARCODER', 'LLAMA_2_70B_CHAT', 'GRANITE_13B_INSTRUCT', 'GRANITE_13B_CHAT']


You need to specify `model_id` that will be used for inferencing:

ในการให้ LLM ทำงานนั้น เราเลือก `model_id` ที่เราต้องการจะใช้มัน:

In [57]:
model_id = ModelTypes.GRANITE_13B_CHAT

### Defining the model parameters

You might need to adjust model `parameters` for different models or tasks, to do so please refer to documentation under `GenTextParamsMetaNames` class.

**Action:** If any complications please refer to the [documentation](https://ibm.github.io/watson-machine-learning-sdk/).



### ตั้งค่า parameters ของ model

เราอาจต้องการ การปรับ `parameters` สำหรับ โมเดลที่ต่างกัน เพื่อทำประเภทของงานที่ต่างกัน สามารถอ่านข้อมูลเพิ่มเติมได้ที่ `GenTextParamsMetaNames` class
[documentation](https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html#metanames.GenTextParamsMetaNames).

In [58]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.MIN_NEW_TOKENS: 0,
    GenParams.MAX_NEW_TOKENS: 1,
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.REPETITION_PENALTY: 1
}

### Initialize the model
Initialize the `Model` class with previous set params.

เลือก `Model` และตั้งค่า `parameters`

In [59]:
from ibm_watson_machine_learning.foundation_models import Model

model = Model(
    model_id=model_id, 
    params=parameters, 
    credentials=credentials,
    project_id=project_id)

### Model's details

In [60]:
model.get_details()

{'model_id': 'ibm/granite-13b-chat-v1',
 'label': 'granite-13b-chat-v1',
 'provider': 'IBM',
 'source': 'IBM',
 'short_description': 'The Granite model series is a family of IBM-trained, dense decoder-only models, which are particularly well-suited for generative tasks.',
 'long_description': 'Granite models are designed to be used for a wide range of generative and non-generative tasks with appropriate prompt engineering. They employ a GPT-style decoder-only architecture, with additional innovations from IBM Research and the open community.',
 'task_ids': ['question_answering',
  'summarization',
  'classification',
  'generation',
  'extraction'],
 'tasks': [{'id': 'question_answering', 'ratings': {'quality': 3}},
  {'id': 'summarization', 'ratings': {'quality': 2}},
  {'id': 'retrieval_augmented_generation', 'ratings': {'quality': 2}},
  {'id': 'classification', 'ratings': {'quality': 3}},
  {'id': 'generation'},
  {'id': 'extraction', 'ratings': {'quality': 2}}],
 'model_limits': {

<a id="predict"></a>
## Analyze the satisfaction

#### Prepare prompt and generate text

#### เตรียม prompt และ เริ่มสร้างคำ

In [61]:
instruction = """Determine if the customer was satisfied with the experience based on the comment. Return simple yes or no.
Comment:The car was broken. They couldn't find a replacement. I've waster over 2 hours.
Satisfied:no"""

In [62]:
prompt1 = "\n".join([instruction, "Comment:" + comments[2], "Satisfied:"])
print(prompt1)

Determine if the customer was satisfied with the experience based on the comment. Return simple yes or no.
Comment:The car was broken. They couldn't find a replacement. I've waster over 2 hours.
Satisfied:no
Comment:My last car rental was in Costa Rica.  There was lots of red tape associated with renting the car, it took almost an hour.  I don't usually experience that type of problem in the US.
Satisfied:


Analyze the sentiment for a sample of zero-shot input from the test set.

In [63]:
print(model.generate_text(prompt=prompt1))

no


### Calculate the accuracy

In [64]:
sample_size = 10
prompts_batch = ["\n".join([instruction, "Comment:" + comment, "Satisfied:"]) for comment in comments[:10]]
results = model.generate_text(prompt=prompts_batch)

In [65]:
print(prompts_batch[0] + results[0])

Determine if the customer was satisfied with the experience based on the comment. Return simple yes or no.
Comment:The car was broken. They couldn't find a replacement. I've waster over 2 hours.
Satisfied:no
Comment:They were helpful and accommodating when I wanted to change the color of the car.
Satisfied:yes


In [54]:
from sklearn.metrics import accuracy_score

label_map = {0: "no", 1: "yes"}
y_true = [label_map[sat] for sat in satisfaction][:sample_size]

print('accuracy_score', accuracy_score(y_true, results))

accuracy_score 0.8


In [66]:
print('true', y_true, '\npred', results)

true ['no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes'] 
pred ['yes', 'no', 'no', 'yes', 'yes', 'no', 'yes', 'yes', 'no', 'no']


# Thai Language version

We have shown how sentiment classification works for the English language, now we will also showcase the same for Thai language.

In [67]:
data_th = pd.read_csv('th_car_rental.csv')

In [68]:
model_id = ModelTypes.LLAMA_2_70B_CHAT #(0.7)
model = Model(
    model_id=model_id, 
    params=parameters, 
    credentials=credentials,
    project_id=project_id)

In [69]:
# train, test = train_test_split(data_th, test_size=0.2)
comments = list(data_th.Customer_Service_TH)
satisfaction = list(data_th.Satisfaction)

In [70]:
instruction = """Determine if the customer was satisfied with the experience based on the comment. Return simple yes or no.
Comment: การบริการลูกค้าก็ดี พวกเขาสามารถอัพเกรดชั้นเรียนถัดไปให้ฉันได้ในราคา $2 ต่อวัน...นั่นเป็นสิ่งที่ดี
Satisfied:yes
Comment: พวกเขามีรถคลาสที่ฉันสั่ง
Satisfied:yes
Comment: มันสบายดี
Satisfied:yes
Comment: ต่อแถวยาวเพื่อรอรับการเช่า
Satisfied:no
Comment: ราคาสูงเกินไปและทำเลอยู่นอกสถานที่ไกลเกินไป
Satisfied:no
Comment: ยังไม่ได้เช่ารถตั้งแต่เดือนตุลาคมปีที่แล้ว บริการลูกค้าได้ดี มีบริการรถรับส่งไปและกลับจากสนามบิน การมารับตามกำหนดเวลานั้นตรงเวลาและพวกเขาก็พบพัสดุที่เราทิ้งไว้ในรถและนำไปที่รถรับส่งด้วย
Satisfied:yes
Comment: ไม่มีปัญหา
Satisfied:yes
Comment: พนักงานก็น่าพอใจมาก ให้แผนที่/เส้นทางไปยังโรงแรมของฉัน
Satisfied:yes
Comment: มีปัญหาบางอย่าง รถของฉันไม่พร้อมตรงเวลา
Satisfied:no
Comment: การบริการลูกค้าก็ดี พวกเขาสามารถอัพเกรดชั้นเรียนถัดไปให้ฉันได้ในราคา $2 ต่อวัน...นั่นเป็นสิ่งที่ดี
Satisfied:yes
"""

In [71]:
prompt1 = "\n".join([instruction, "Comment:" + comments[3], "Satisfied:"])
print(prompt1)

Determine if the customer was satisfied with the experience based on the comment. Return simple yes or no.
Comment: การบริการลูกค้าก็ดี พวกเขาสามารถอัพเกรดชั้นเรียนถัดไปให้ฉันได้ในราคา $2 ต่อวัน...นั่นเป็นสิ่งที่ดี
Satisfied:yes
Comment: พวกเขามีรถคลาสที่ฉันสั่ง
Satisfied:yes
Comment: มันสบายดี
Satisfied:yes
Comment: ต่อแถวยาวเพื่อรอรับการเช่า
Satisfied:no
Comment: ราคาสูงเกินไปและทำเลอยู่นอกสถานที่ไกลเกินไป
Satisfied:no
Comment: ยังไม่ได้เช่ารถตั้งแต่เดือนตุลาคมปีที่แล้ว บริการลูกค้าได้ดี มีบริการรถรับส่งไปและกลับจากสนามบิน การมารับตามกำหนดเวลานั้นตรงเวลาและพวกเขาก็พบพัสดุที่เราทิ้งไว้ในรถและนำไปที่รถรับส่งด้วย
Satisfied:yes
Comment: ไม่มีปัญหา
Satisfied:yes
Comment: พนักงานก็น่าพอใจมาก ให้แผนที่/เส้นทางไปยังโรงแรมของฉัน
Satisfied:yes
Comment: มีปัญหาบางอย่าง รถของฉันไม่พร้อมตรงเวลา
Satisfied:no
Comment: การบริการลูกค้าก็ดี พวกเขาสามารถอัพเกรดชั้นเรียนถัดไปให้ฉันได้ในราคา $2 ต่อวัน...นั่นเป็นสิ่งที่ดี
Satisfied:yes

Comment:ฉันไม่ได้รับรถที่ฉันบอกว่าจะว่าง มีค่าธรรมเนียมซ่อนเร้นเพิ่มเ

In [72]:
print(model.generate_text(prompt=prompt1))

no


In [73]:
sample_size = 10
prompts_batch = ["\n".join([instruction, "Comment:" + comment, "Satisfied:"]) for comment in comments[:10]]
results = model.generate_text(prompt=prompts_batch)

In [74]:
label_map = {0: "no", 1: "yes"}
y_true = [label_map[sat] for sat in satisfaction][:sample_size]

print('accuracy_score', accuracy_score(y_true, results))

accuracy_score 0.8


<a id="summary"></a>
## Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to analyze car rental customer satisfaction with watsonx.ai foundation model.
 
 Check out our _[Online Documentation]()_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Mateusz Szewczyk**, Software Engineer at Watson Machine Learning.

**Lukasz Cmielowski**, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2023 IBM. This notebook and its source code are released under the terms of the MIT License.